## Railroad oriented programming with BoundResult - 한국어

파이썬은 예외를 통해 작업의 성패를 관리하지만,
함수형 프로그래밍 언어들은 railroad oriented programming(이하 ROP)이라는 재미있는 개념을 통해 Exception을 대체합니다.

fieldenum은 파이썬에 잘 어울리는 ROP를 위한 `BoundResult`라는 enum을 `fieldenum.enums` 모듈에서 제공하고 있습니다.

<!-- > [!CAUTION] -->
> `fieldenums.enums` 모듈은 현재는 파이썬 3.12 이상에서만 사용 가능합니다. 향후에는 지원이 확대될 수 있습니다.

이 문서에서는 `BoundResult`를 통한 파이썬에서의 ROP를 구현하는 방식을 설명합니다.

## 기초

다음과 같은 함수가 있다고 이야기해 봅시다.

In [2]:
from fieldenum import Unit, Variant, fieldenum, unreachable
from fieldenum.enums import BoundResult, Message, Option


def calculate(value: str) -> float:
    int_value = int(value)
    return 100 / int_value

이 함수는 문자열로 값을 받아서 정수로 변환한 다음에 100에서 변환된 값을 나눈 뒤 값을 내보내는 함수입니다.
다음과 같이 사용할 수 있습니다.


In [2]:
assert calculate("5") == 20.0
assert calculate("10") == 10.0
assert calculate("100") == 1.0
assert calculate("500") == 0.2

이 함수는 두 가지 상황에서 오류를 내보낼 가능성이 있습니다.
첫 번째는 `value`를 `int`로 변환하는 데에 실패하면 `ValueError`가 발생할 가능성이 있고,
두 번쩨는 `int_value`가 `0`이 되면 `ZeroDivisionError`가 발생할 가능성이 있습니다.

In [3]:
calculate("not_an_integer")  # ValueError

ValueError: invalid literal for int() with base 10: 'not_an_integer'

In [4]:
calculate("0")  # ZeroDivisionError

ZeroDivisionError: division by zero

예외를 이용해 잘못된 값을 처리하는 것은 장점도 있지만 단점도 명확합니다.
우선 사용자는 해당 함수가 어떤 오류를 내보낼지에 대해 알 수가 없습니다.
또한 개발자도 사용자가 함수의 오류를 적절하게 처리했는지를 알 수 있는 방법이나 처리하도록 강제할 수 있는 방법이 없습니다.

ROP는 다른 접근을 취합니다.
예외라는 또다른 제어 흐름을 만드는 대신, 성공과 실패라는 두 가지 상태를 가질 수 있는 하나의 타입을 리턴하는 방식을 사용합니다.
사용자가 값을 사용하려면, 해당 타입을 직접 처리해 오류를 해결해야 합니다.

한 번 `BoundResult`를 이용해서 구현해 보죠.
`BoundResult`는 `Success`와 `Failed` 두 상태를 지닙니다.
(두 번째 인자에 들어가는 값에 대해서는 조금 이따가 설명해 드리겠습니다.)

In [5]:
print(BoundResult.Success("operation success!", Exception))  # 성공을 나타냄
print(BoundResult.Failed(ValueError("operation failed..."), Exception))  # 실패를 나타냄

BoundResult.Success('operation success!', <class 'Exception'>)
BoundResult.Failed(ValueError('operation failed...'), <class 'Exception'>)


파이썬 함수를 원래 예외를 raise해 성패를 알리지만,
ROP를 사용하는 함수는 그 대신 `Success`와 `Failed`를 통해 값을 보냄으로써 성패를 알립니다.

In [3]:
def ordinary_python_function(value) -> str:
    if isinstance(value, str):
        return value + "world!"  # valid operation returns.
    else:
        raise TypeError("Type of value is invalid")  # invalid operation raises.

def rop_function(value) -> BoundResult[str, Exception]:
    if isinstance(value, str):
        return BoundResult.Success(value + "world!", Exception)  # valid operation returns Success.
    else:
        return BoundResult.Failed(TypeError("Type of value is invalid"), Exception)  # invalid operation returns Failed.

이렇게 하면 사용자는 이제 오류가 일어나지 않을 거라고 넘겨짚을 수 없습니다. 반드시 어떠한 방식으로든 명시적으로 예외를 처리해야 합니다.

In [ ]:
result1: str = ordinary_python_function(1233)
print(result1)  # 작동하지 않더라도 아무튼 사용할 수도 있습니다

In [6]:
result2: BoundResult[str, Exception] = rop_function(123)

match result2:  # 명시적으로 값을 처리하지 않으면 사용할 수 없습니다.
    case BoundResult.Success(value, _):
        print(value)

    case BoundResult.Failed(err, _):
        print("failed...")

failed...


근데 그러면 모든 함수를 저렇게 못생기고 일반적인 모습에서 벗어나는 방식으로 짜야 할까요?

물론 아닙니다. `BoundResult.wrap` 데코레이터를 통해 값을 내보내는 일반적인 함수를 `BoundResult`를 내보내는 함수로 변환할 수 있습니다.

In [8]:
@BoundResult.wrap(ArithmeticError)
def calculate(value: str) -> float:
    int_value = int(value)
    return 100 / int_value

이제 `calculate`는 `@BoundResult.wrap(Exception)`로 감싸졌습니다.
이제 한 번 다시 `calculate`를 사용해 봅시다.

In [9]:
calculate("5")

BoundResult.Success(20.0, <class 'ArithmeticError'>)

함수값 `20.0`이 그대로 리턴되는 대신 `BoundResult.Success`라는 값으로 감싸진 것을 확인할 수 있습니다.
이는 이 함수가 예외를 일으키지 않고 정상적으로 값을 반환했다는 의미입니다.

에제 한 번 함수가 실패하도록 해 볼까요? 다음과 같은 코드는 원래 예외를 일으켜야 합니다.

In [10]:
calculate("0")

BoundResult.Failed(ZeroDivisionError('division by zero'), <class 'ArithmeticError'>)

예외가 일어나는 대신 `BoundResult.Failed`라는 배리언트가 리턴되었네요.
이는 `BoundResult.wrap`으로 감싸진 함수가 예외를 일으킬 경우 예외를 잡아 `BoundResult.Failed`의 값으로 만들어 리턴합니다.

좋네요. 그렇지만 특정 오류는 리턴으로 처리되는 대신 그냥 예외로 던져지는 것이 나을 수도 있습니다. 예를 들어 `KeyboardInterrupt`나 `SystemExit`같은 오류는 굳이 잡기보단 원래 자기가 하던 일을 할 수 있도록 오류가 전파되는 것이 더 좋을 수 있습니다.

따라서 사용자는 bound를 명시적으로 설정해야 합니다. 이는 예외가 전파되지 않을 기준을 설정합니다. 예를 들어 우리의 `calculate` 함수는 `ArithmeticError`를 바운드로 설정했는데, `ZeroDivisionError`는 `ArithmeticError`의 서브클래스이므로 예외가 전파되는 대신 `BoundResult.Failed`가 리턴됩니다.
하지만 `int` 함수는 변환에 실패했을 경우 `ValueError`를 내보내고, 이는 `ArithmeticError`의 서브클래스가 아니기 때문에 이는 오류로 전파됩니다.

In [11]:
calculate("not_an_integer")

ValueError: invalid literal for int() with base 10: 'not_an_integer'

마지막으로 `BoundResult.Success`와 `BoundResult.Failed`는 모두 `fieldenum.enums`에서 직접적으로 접근이 가능합니다.
따라서 다음과 같이 임포트해서 사용할 수도 있습니다.

In [12]:
from fieldenum.enums import BoundResult, Success, Failed  # noqa: I001

assert Success(1000, Exception) == BoundResult.Success(1000, Exception)

error = ValueError()
assert Failed(error, Exception) == BoundResult.Failed(error, Exception)

## BoundResult의 메서드 사용하기

`BoundResult`는 더욱 간단한 방식으로 ROP를 실현하기 위한 다양한 메서드를 지원합니다.

In [13]:
# 이 장의 모든 코드들은 이 코드들이 먼저 실행된다고 가정하겠습니다.

from fieldenum.enums import BoundResult, Success, Failed, Some  # noqa: I001

@BoundResult.wrap(ArithmeticError)
def calculate(value: str) -> float:
    int_value = int(value)
    return 100 / int_value

### `BoundResult.unwrap([default])`

기본적으로는 오류를 처리하기 위해서는 match문을 사용해야 합니다.

In [14]:
match calculate("5"):
    case Success(ok, _):
        print(ok)

    case Failed(err, _):
        raise err

20.0


In [15]:
match calculate("0"):
    case Success(ok, _):
        print(ok)

    case Failed(err, _):
        raise err

ZeroDivisionError: division by zero

하지만 모든 오류마다 이러한 코드를 작성하기에는 힘이 들 겁니다. 따라서 `.unwrap()`을 통해 한 줄로 해당 코드와 동일한 작업을 하는 코드를 생성할 수 있습니다.

In [16]:
calculate("5").unwrap()

20.0

In [17]:
calculate("0").unwrap()

ZeroDivisionError: division by zero

러스트의 경우에는 `.unwrap_or(default)`라는 메서드를 통해 값이 실패일 때 기본값을 제공할 수 있습니다.

이와 비슷하게 `.unwrap(default)`와 같이 `.unwrap` 메서드에 기본값을 제공하면 `BoundResult.Failed`일 경우 `default`를 리턴합니다.

In [18]:
calculate("5").unwrap(0.0)  # 기본값이 사용되지 않고 함수의 반환값이 unwrap됩니다.

20.0

In [19]:
calculate("0").unwrap(0.0)  # 결과가 실패였기 때문에 기본값이 사용됩니다.

0

### `bool()`

`BoundResult`는 성공일 때는 `True`로 간주되고, 실패일 때는 `False`로 간주됩니다. 이 방식을 통해서도 간단하고 확실하게 오류인지 검증할 수 있습니다.

In [20]:
if calculate("5"):
    print("success!")
else:
    print("failed...")

success!


In [21]:
if calculate("0"):
    print("success!")
else:
    print("failed...")

failed...


### `BoundResult.map(func)`

`.map(func)` 메서드는 `BoundResult.Success(value)`일 경우에는 `func(value)`를 실행하고 그 값을 다시 `BoundResult`에 넘깁니다.
하지만 값이 `BoundResult.Failed`일 경우에는 아무것도 실행하지 않고 그대로를 넘깁니다.

In [22]:
def revert(x: float) -> float:
    return 1 / x

calculate("200").map(revert)  # 결과값인 0.5가 역수가 되어 2.0이 됩니다.

BoundResult.Success(2.0, <class 'ArithmeticError'>)

In [23]:
Success(0, ArithmeticError).map(revert)  # map의 함수가 실패했을 때는 `BoundResult.Failed`가 리턴됩니다.

BoundResult.Failed(ZeroDivisionError('division by zero'), <class 'ArithmeticError'>)

In [24]:
calculate("0").map(revert)  # `revert`가 수행되지 않고 기존의 Failed 배리언트가 다시 건네집니다.

BoundResult.Failed(ZeroDivisionError('division by zero'), <class 'ArithmeticError'>)

이때 함수가 `BoundResult`를 반환한다면 해당 값을 기준으로 성패를 결정합니다. 만약 `BoundResult`가 반환되었을 때조차도 무조건 성공으로 처리하고 싶다면 `as_is`를 `True`로 두세요.

In [25]:
@BoundResult.wrap(ArithmeticError)
def revert(x: float) -> float:
    return 1 / x

Success(0, ArithmeticError).map(revert)  # revert가 예외 대신 BoundResult를 리턴하지만 잘 처리합니다.

BoundResult.Failed(ZeroDivisionError('division by zero'), <class 'ArithmeticError'>)

In [26]:
# as_is를 True로 두면 revert 함수가 `BoundResult`를 리턴하면 성공한 것으로 처리하고 그 값을 그대로 값으로 삼습니다.
# 일반적으로는 사용할 일이 없습니다.
Success(0, ArithmeticError).map(revert, as_is=True)

BoundResult.Success(BoundResult.Failed(ZeroDivisionError('division by zero'), <class 'ArithmeticError'>), <class 'ArithmeticError'>)

### `BoundResult.as_option()`

아까 설명한 `enums` 모듈에는 사실 `Option`이라는 다른 enum도 포함되어 있습니다.

어떨 때는 `BoundResult`가 `Option`으로 변환되는 것이 필요할 수 있습니다.

그럴 때에 위해, `.as_option()` 메서드는 `BoundResult.Success`일 때는 `Option.Some`을, `BoundResult.Failed`일 때는 `Option.Nothing`을 내보냅니다.

In [27]:
calculate("5").as_option()

Option.Some(20.0)

In [28]:
calculate("0").as_option()

Option.Nothing

### `BoundResult.rebound(Bound)`

바운드를 수정할 때는 `.rebound(Bound)`를 사용할 수 있습니다.

In [29]:
calculate("5").rebound(Exception)

BoundResult.Success(20.0, <class 'Exception'>)

이때 새로운 `BoundResult` 객체가 만들어지고, 기존의 값은 변경되지 않는다는 사실에 주의하세요.

In [30]:
result = calculate("5")
print(result)
print(result.rebound(Exception))  # rebound가 새로운 객체를 반환합니다.
print(result)  # 기존에 `result`에 있던 객체는 변환되지 않습니다.

BoundResult.Success(20.0, <class 'ArithmeticError'>)
BoundResult.Success(20.0, <class 'Exception'>)
BoundResult.Success(20.0, <class 'ArithmeticError'>)


## 잇기 심화

`BoundResult`는 결과를 서로 잇기 위한 별도의 연산자를 제공하지는 않습니다. 하지만 다양한 기본 연산자들과 함수들을 통해 결과를 취합할 수 있습니다.

다음과 같은 두 함수가 있다고 해봅시다.
`will_fail`는 이름처럼 항상 실패하고, `will_success`는 항상 성공합니다.
단, 이 값들은 모두 `BoundResult.Failed`와 `BoundResult.Success`로 리턴됩니다.

In [31]:
@BoundResult.wrap(Exception)
def will_fail(value: int) -> float:
    raise ValueError(value)

@BoundResult.wrap(Exception)
def will_success(value: int) -> int:
    return value

우선 두 개의 연산 중 하나라도 실패한 게 있는지 확인하고 싶다면 `and`를 통해 결과를 이어 확인할 수 있습니다.

In [32]:
will_fail(1) and will_success(2)  # 한 결과가 Failed이므로 해당 값을 리턴합니다.

BoundResult.Failed(ValueError(1), <class 'Exception'>)

In [33]:
will_success(1) and will_success(2)  # 모든 결과가 Success이므로 성공을 리턴합니다.

BoundResult.Success(2, <class 'Exception'>)

In [34]:
will_success(1) and will_success(2) and will_success(3) and will_success(4)  # 임의의 길이에서도 작동합니다.

BoundResult.Success(4, <class 'Exception'>)

In [35]:
will_success(1) and will_success(2) and will_fail(3) and will_success(4)

BoundResult.Failed(ValueError(3), <class 'Exception'>)

반대로 전체 중 하나라도 성공한 게 있는지 확인하려면 `or`을 사용할 수 있습니다. 마찬가지로 임의의 길이로 사용할 수 있습니다.

In [36]:
will_fail(1) or will_success(2)

BoundResult.Success(2, <class 'Exception'>)

In [37]:
will_fail(1) or will_fail(2) or will_fail(3) or will_fail(4)

BoundResult.Failed(ValueError(4), <class 'Exception'>)

In [38]:
will_fail(1) or will_fail(2) or will_success(3) or will_fail(4)

BoundResult.Success(3, <class 'Exception'>)

만약 결과의 개수가 임의적이거나 너무 많아 고정된 길이로 연결하기 어렵다면 `key=bool`로 설정한 `max`나 `min`을 이용할 수도 있습니다.

In [39]:
max((will_fail(i) for i in range(100)), key=bool)  # max에서 모든 함수가 실패했으므로 가장 처음에 실패한 값을 내보냅니다.

BoundResult.Failed(ValueError(0), <class 'Exception'>)

In [40]:
max(*(will_fail(i) for i in range(100)), will_success(100), key=bool)  # max에서 한 함수가 성공했으므로 해당 값을 내보냅니다.

BoundResult.Success(100, <class 'Exception'>)

In [41]:
min(*(will_success(i) for i in range(100)), will_fail(100), key=bool)  # min에서 한 함수가 실패했으므로 해당 값을 내보냅니다.

BoundResult.Failed(ValueError(100), <class 'Exception'>)

만약 결과가 필요하지 않고 성패 여부만 궁금하다면 `all`과 `any`를 사용할 수 있습니다.

* `all`: 모든 함수가 성공했다면 `True`, 하나라도 실패했다면 `False`를 리턴합니다.
* `any`: 한 함수라도 성공했다면 `True`, 모든 함수가 실패했다면 `False`를 리턴합니다.

기존의 `all`과 `any`의 뜻과 일치하므로 그리 어렵지 않게 받아들이실 수 있을 것입니다.
또한 이 함수들은 lazy하므로 `min`, `max`보다 살짝 더 효율적입니다.

In [42]:
all(will_success(i) for i in range(100))  #  모두 성공했으므로 True를 리턴합니다.

True

In [43]:
# 하나가 실패했으므로 False를 리턴합니다. 이 과정에서 뒤쪽을 일일이 확인하지 않고 `will_fail`을 만나자마자 바로 리턴합니다.
all((will_fail(-1), *(will_success(i) for i in range(100))))

False

In [44]:
any(will_fail(i) for i in range(100))  #  모두 실패했으므로 False를 리턴합니다.

False

In [45]:
# 하나가 성공했으므로 True를 리턴합니다. 이 과정에서 뒤쪽을 일일이 확인하지 않고 `will_success`을 만나자마자 바로 리턴합니다.
any((will_success(-1), *(will_fail(i) for i in range(100))))

True